In [17]:
out = """
args: Namespace(model_output='out/', train=False, train_dataset=None, evaluate=True, model_input='/scratch/pw1329/nlu-s2021-final-project/models/sbc-model-new/', no_cuda=False, batch_size=2, eval_dataset='ted_talks_iwslt', eval_all=False)
predictions: [tensor([-0.8143, -1.0416,  1.1337]), tensor([-0.3493, -0.6984,  0.8503]), tensor([ 0.1069, -0.8112,  0.8317]), tensor([ 0.7548, -0.5824,  0.4057])]
ayy lmao | tensor([-0.8143, -1.0416,  1.1337])
ayy lmao2 | tensor([-0.8143, -1.0416,  1.1337])
ayy lmao3 | tensor([-0.8143, -1.0416,  1.1337])

"""

In [18]:
import pandas as pd

In [26]:
from io import StringIO

In [27]:
outfile = StringIO(out)

In [28]:
df = pd.read_csv(outfile, delimiter="|", skiprows=3, names=["sentence", "predictions"])

In [29]:
df

,sentence,predictions
0,ayy lmao,"tensor([-0.8143, -1.0416, 1.1337])"
1,ayy lmao2,"tensor([-0.8143, -1.0416, 1.1337])"
2,ayy lmao3,"tensor([-0.8143, -1.0416, 1.1337])"


In [30]:
df["predictions"].apply(lambda x: x.replace("tensor(", "").replace(")", ""))

0     [-0.8143, -1.0416,  1.1337]
1     [-0.8143, -1.0416,  1.1337]
2     [-0.8143, -1.0416,  1.1337]
Name: predictions, dtype: object

In [37]:
df["new_preds"] = df["predictions"].apply(lambda x: x.replace("tensor(", "").replace(")", ""))

In [43]:
df.join(pd.DataFrame(df["new_preds"].apply(tensor_split_mapper).tolist()))

,sentence,predictions,new_preds,prediction0,prediction1,prediction2
0,ayy lmao,"tensor([-0.8143, -1.0416, 1.1337])","[-0.8143, -1.0416, 1.1337]",-0.8143,-1.0416,1.1337
1,ayy lmao2,"tensor([-0.8143, -1.0416, 1.1337])","[-0.8143, -1.0416, 1.1337]",-0.8143,-1.0416,1.1337
2,ayy lmao3,"tensor([-0.8143, -1.0416, 1.1337])","[-0.8143, -1.0416, 1.1337]",-0.8143,-1.0416,1.1337


In [56]:
# for number in [123, 123, 123]:
# create column named prediction0, prediction1, prediction2
# new columns names = range(len(eval(tensor string)))
# apply for all N

def tensor_split_mapper(input_tensor):
    # return a dictionary for each row 
    input_parsed_tensor = eval(input_tensor)
    prediction_counts = len(input_parsed_tensor)
    out = {}
    for prediction_num in range(0, prediction_counts):
        out[f"prediction{prediction_num}"] = input_parsed_tensor[prediction_num]
    return out
    



def read_outfile(outfile_name, delimiter="|", skiprows=2, split_tensor=True):
    df = pd.read_csv(outfile_name, delimiter=delimiter, skiprows=skiprows, names=["sentence", "predictions"])
    df["predictions"] = df["predictions"].apply(lambda x: x.replace("tensor(", "").replace(")", ""))
    if split_tensor:
        df = df.join(pd.DataFrame(df["predictions"].apply(tensor_split_mapper).tolist()))
    return df

In [57]:
tensor_split_mapper("[123, 234, 678]")

{'prediction0': 123, 'prediction1': 234, 'prediction2': 678}

In [59]:
df = read_outfile("dummy.out")

In [63]:
df.describe()

,prediction0,prediction1,prediction2
count,5.0000,5.0000,5.000000
mean,-0.8143,-1.0416,22288.480220
std,0.0000,0.0000,49779.653187
min,-0.8143,-1.0416,0.000000
25%,-0.8143,-1.0416,1.133700
50%,-0.8143,-1.0416,6.133700
75%,-0.8143,-1.0416,98.133700
max,-0.8143,-1.0416,111337.000000


,prediction0,prediction1,prediction2
prediction0,0.0,0.0,0.000000e+00
prediction1,0.0,0.0,0.000000e+00
prediction2,0.0,0.0,2.478014e+09
